<a href="https://colab.research.google.com/github/Vatsav-the-learner/LLM-s/blob/main/Classification/Labeled_Data_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers torch
!pip install transformers sentence-transformers openai
!pip install -U datasets==2.14.6


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# 1. Load the dataset
# AG News has 4 categories: 0-World, 1-Sports, 2-Business, 3-Sci/Tech
from datasets import load_dataset
ag_news_dataset = load_dataset("ag_news", cache_dir=None)
ag_news_dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [ ]:
import torch
print(torch.cuda.is_available())


True


In [ ]:
# 2. Load a pre-trained LLM pipeline for text classification
# We use a model fine-tuned on sentiment analysis (SST-2) for this simple example.
# It will classify text as 'POSITIVE' or 'NEGATIVE'.
# Note: For best results, you'd fine-tune a model on the ag_news data itself.
from transformers import pipeline
#model_path = "distilbert-base-uncased-finetuned-sst-2-english"
model_path = "textattack/distilbert-base-uncased-ag-news"
#model_path = "bert-base-uncased"
#classifier = pipeline("text-classification", model=model_path)
classifier = pipeline(model=model_path,  tokenizer = model_path, return_all_scores=True)
#classifier = pipeline("feature-extraction",model=model_path,  tokenizer = model_path)


config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

# Run inference
y_pred = []
for output in tqdm(classifier(KeyDataset(ag_news_dataset["test"], "text")), total=len(ag_news_dataset["test"])):
    # 'output' will be a list of 4 dictionaries for 'LABEL_0' to 'LABEL_3'
    # e.g., [{'label': 'LABEL_0', 'score': 0.05}, {'label': 'LABEL_1', 'score': 0.01}, {'label': 'LABEL_2', 'score': 0.92}, {'label': 'LABEL_3', 'score': 0.02}]

    # Extract all 4 scores
    scores = [item["score"] for item in output]

    # Find the index of the highest score (which corresponds to the predicted class)
    assignment = np.argmax(scores)
    y_pred.append(assignment)

100%|██████████| 7600/7600 [00:45<00:00, 165.60it/s]


In [ ]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
    """Create and print the classification report for AG News."""
    ag_news_target_names = ["World", "Sports", "Business", "Sci/Tech"]
    performance = classification_report(
        y_true, y_pred,
        target_names=ag_news_target_names, # Modified target_names
        digits=4 # Added for more precision, optional
    )
    print(performance)

In [ ]:
evaluate_performance(ag_news_dataset["test"]["label"], y_pred)

              precision    recall  f1-score   support

       World     0.9692    0.9453    0.9571      1900
      Sports     0.9849    0.9926    0.9887      1900
    Business     0.9347    0.9047    0.9195      1900
    Sci/Tech     0.9047    0.9489    0.9263      1900

    accuracy                         0.9479      7600
   macro avg     0.9484    0.9479    0.9479      7600
weighted avg     0.9484    0.9479    0.9479      7600

